<a href="https://colab.research.google.com/github/SalvatoreTucci/CPV/blob/main/CPV_Text_categorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U spaCy
!python -m spacy download it_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-12-03 15:27:00.386336: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.0 MB 1.8 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')


In [ ]:
import pandas as pd
import string
import spacy
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from spacy.lang.it.stop_words import STOP_WORDS
from spacy.lang.it import Italian
from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn import neural_network
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

#init spaCy
punctuations = string.punctuation
nlp = spacy.load("it_core_news_sm")
stop_words = spacy.lang.it.stop_words.STOP_WORDS
parser = Italian()

# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

# Tokenizer function
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.text for word in mytokens ]
    # remove stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    # return preprocessed list of tokens
    return mytokens

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import collections

df1 = pd.read_csv('/content/drive/MyDrive/cpv.csv', sep=",", names=["cig","oggetto_bando","oggetto_lotto","oggetto_principale","cpv","desc_cpv"], encoding='utf-8')
df2 = pd.read_csv('/content/drive/MyDrive/cpv_2008.csv', sep=",")
df2 = df2.rename({'cod_cpv':'cpv'}, axis=1)
cpv1 = df1["cpv"]
cpv2 = df2["cpv"]
#BoW with word count
vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))
#BoW with TF-IDF
vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

"""
# join tra dataset cpv e dataset bandi di gara
df3 = pd.merge(cpv2,cpv1,on='cpv', how='left')
cpv3 = df3["cpv"]
# cpv
y = [x[0:10] for x in cpv1]
print(len(collections.Counter(y))) # totale elementi supera totale cpv esistenti -> ci sono cpv inesistenti len(y)
"""

'\n# join tra dataset cpv e dataset bandi di gara\ndf3 = pd.merge(cpv2,cpv1,on=\'cpv\', how=\'left\')\ncpv3 = df3["cpv"]\n# cpv\ny = [x[0:10] for x in cpv1]\nprint(len(collections.Counter(y))) # totale elementi supera totale cpv esistenti -> ci sono cpv inesistenti len(y)\n'

In [ ]:
df1 = df1[df1.duplicated('cpv',keep=False)] # rimuovo i cpv con un solo bando associato

X = df1['oggetto_bando']
y = df1['cpv'] # the labels

In [ ]:
# divisioni
y = [x[0:2] for x in df1['cpv']]
X_main, X_test, y_main, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y) # divisione in main e test 80% - 20%
X_train, X_val, y_train, y_val = train_test_split(X_main, y_main, test_size=0.125, random_state=42, stratify=y_main) # divisione del main in train e valutation 70% - 10%
"""
# gruppi
y = [x[0:3] for x in df1['cpv']]
X_main, X_test, y_main, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_main, y_main, test_size=0.125, random_state=42, stratify=y_main)

# classi
y = [x[0:4] for x in df1['cpv']]
X_main, X_test, y_main, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_main, y_main, test_size=0.125, random_state=42, stratify=y_main)

# categorie
y = [x[0:5] for x in df1['cpv']]
X_main, X_test, y_main, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_main, y_main, test_size=0.125, random_state=42, stratify=y_main)

# cpv
y = [x[0:10] for x in df1['cpv']]
X_main, X_test, y_main, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_main, y_main, test_size=0.125, random_state=42, stratify=y_main)
"""

"\n# gruppi\ny = [x[0:3] for x in df1['cpv']]\nX_main, X_test, y_main, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)\nX_train, X_val, y_train, y_val = train_test_split(X_main, y_main, test_size=0.125, random_state=42, stratify=y_main)\n\n# classi\ny = [x[0:4] for x in df1['cpv']]\nX_main, X_test, y_main, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)\nX_train, X_val, y_train, y_val = train_test_split(X_main, y_main, test_size=0.125, random_state=42, stratify=y_main)\n\n# categorie\ny = [x[0:5] for x in df1['cpv']]\nX_main, X_test, y_main, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)\nX_train, X_val, y_train, y_val = train_test_split(X_main, y_main, test_size=0.125, random_state=42, stratify=y_main)\n\n# cpv\ny = [x[0:10] for x in df1['cpv']]\nX_main, X_test, y_main, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)\nX_train, X_val, y_train, y_val = train_test_split(X_main

In [ ]:
# Linear Support Vector Machine classifier
classifier = svm.LinearSVC()
# Multi-layer Perceptron classifier
#classifier = neural_network.MLPClassifier()
# Create the pipeline
pipe = Pipeline([("cleaner", predictors()),
('vectorizer', vector),
('classifier', classifier)])
# Fit the model
pipe.fit(X_train, y_train)

Pipeline(steps=[('cleaner', <__main__.predictors object at 0x7feda01d0ee0>),
                ('vectorizer',
                 TfidfVectorizer(tokenizer=<function spacy_tokenizer at 0x7feda0d401f0>)),
                ('classifier', LinearSVC())])

In [ ]:
print(pipe.score(X_test, y_test))
predicted = pipe.predict(X_test)
acc = metrics.accuracy_score(y_test, predicted)
print("Accuracy:", acc)
precision = metrics.precision_score(y_test, predicted, average='macro')
recall = metrics.recall_score(y_test, predicted, average='macro')
fm = metrics.f1_score(y_test, predicted, average='macro')
print("P={0}, R={1}, F1={2}".format(precision, recall, fm))
print("======================================================")
print(metrics.classification_report(y_test, predicted, digits=4))

0.768348455184605
Accuracy: 0.768348455184605


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


P=0.5308846661148918, R=0.3787464673661147, F1=0.40887691615100585


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          01     0.3333    0.0312    0.0571        32
          02     0.5000    0.0625    0.1111        16
          03     0.7003    0.6145    0.6546      4412
          05     0.0000    0.0000    0.0000         6
          09     0.7763    0.7551    0.7656     21231
          10     0.0000    0.0000    0.0000         2
          11     0.7500    0.1463    0.2449        82
          13     0.0000    0.0000    0.0000         3
          14     0.5914    0.3875    0.4682      2854
          15     0.8136    0.7973    0.8054     12120
          16     0.5866    0.4201    0.4896      1371
          17     0.2000    0.0345    0.0588        58
          18     0.7097    0.7161    0.7129      9224
          19     0.5967    0.4038    0.4816      3752
          20     0.3333    0.0227    0.0426        44
          21     0.6316    0.0857    0.1509       140
          22     0.7356    0.7265    0.7310     38089
          23     0.6818    

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
fmicro = 0
fmacro = 0
X.iloc[[31829, 68724, 78173, 87799, 182204, 186082, 241863, 267822, 271924, 286483, 292493, 325122, 353594, 353602, 353769, 353788, 353824, 353974, 354280, 354422, 354425, 354463, 354472, 354477, 354597, 354657, 354704, 354738, 354768, 354772, 354793, 354826, 354827, 354847, 355159, 355242, 355362, 355599, 355661, 355704, 355722, 355729, 355763, 355802, 355860, 355944, 355948, 356057, 356149, 356318, 356623, 356943, 356999, 357067, 357073, 357118, 357202, 357248, 357648, 357671, 357716, 357749, 357914, 358163, 358382, 358396, 358416, 358811, 359283, 359583, 360200, 360268, 360444, 360458, 360566, 360630, 360693, 360725, 360962, 361087, 361449, 361749, 361850, 361859, 362021, 362237, 362347, 362362, 362726, 362762, 362774, 363232, 363279, 363280, 363501, 363648, 363659, 363837, 363902, 364155, 364162, 364289, 364385, 364502, 364570, 364677, 364722, 365052, 365085, 365122, 365261, 365326, 365402, 365567, 365609, 365778, 365793, 365831, 365930, 365968, 366036, 366052, 366060, 366091, 527061, 542301, 558383, 595214, 601968, 636511, 676993, 721979, 753300, 769646, 783321, 834494, 913409, 926279, 998202, 998869, 1010924, 1015496, 1029975, 1097487, 1139561, 1191949, 1288512, 1313146, 1314117, 1321210, 1351088, 1369348, 1374542, 1394090, 1411370, 1415237, 1434786, 1440738, 1441511, 1456613, 1469701, 1478890, 1500373, 1551046, 1634460, 1651768, 1684584, 1694683, 1704212, 1735395, 1840028, 1848504, 1853081, 1893443, 1901903, 1953939, 1958814, 1972578, 2024884, 2033010, 2042685, 2062191, 2080868, 2137822, 2147573, 2240110, 2265292, 2269250, 2300596, 2302133, 2302435, 2314729, 2332929, 2393842, 2405060, 2406729, 2437601, 2451603, 2549068, 2596475, 2605994, 2613338, 2634606, 2669522, 2676490, 2684600, 2684729, 2684766, 2684786, 2684842, 2684943, 2685635, 2685688, 2685829, 2685913, 2686403, 2686458, 2686600, 2686611, 2687134, 2687372, 2687412, 2687428, 2687539, 2687555, 2687592, 2687841, 2687986, 2688048, 2688147, 2688284, 2688329, 2688443, 2688597, 2688603, 2688995, 2689013, 2689232, 2689684, 2689773, 2689873, 2689933, 2690053, 2690252, 2690269, 2690373, 2690432, 2690454, 2690496, 2690548, 2690553, 2690626, 2690783, 2690888, 2691007, 2691084, 2691214, 2691241, 2691315, 2691479, 2691668, 2691672, 2691721, 2691832, 2691855, 2691912, 2692017, 2692044, 2692105, 2692359, 2692448, 2692468, 2692572, 2692656, 2693269, 2693328, 2693330, 2693638, 2693690, 2693761, 2693775, 2693808, 2693869, 2693974, 2694048, 2694655, 2694857, 2694886, 2694947, 2695005, 2695121, 2695160, 2695241, 2695401, 2695414, 2695637, 2695722, 2695824, 2695989, 2696066, 2696199, 2696416, 2696487, 2700638, 2728999, 2740168, 2773240, 2789482, 2826505, 2857952, 2877016, 2905587, 2922075, 2923358, 2959758, 2961401, 2967037, 2967935, 2976050, 2989716, 3011459, 3122478, 3158423, 3236394, 3255470, 3316169, 3316364, 3437218, 3547612, 3593934, 3606589, 3618660, 3673279, 3682618, 3682737, 3683108, 3683124, 3683350, 3683581, 3683753, 3683941, 3683999, 3684072, 3684121, 3684131, 3684171, 3684295, 3684479, 3684529, 3684535, 3684536, 3684544, 3684563, 3684610, 3684638, 3684705, 3684983, 3685010, 3685059, 3685116, 3685130, 3685131, 3685137, 3685230, 3685283, 3685347, 3685375, 3685513, 3685756, 3685840, 3685883, 3685976, 3685977, 3686157, 3686227, 3686238, 3686373, 3686384, 3686404, 3686463, 3686689, 3686710, 3686859, 3686881, 3687155, 3687208, 3687271, 3687327, 3687718, 3687729, 3687992, 3688053, 3688057, 3688114, 3688130, 3688147, 3688181, 3688218, 3688239, 3688340, 3688344, 3688487, 3688503, 3688634, 3688661, 3688704, 3688744, 3688831, 3688911, 3689058, 3689258, 3689266, 3689498, 3689565, 3689869, 3690000, 3690463, 3690547, 3690718, 3690798, 3691092, 3691103, 3691197, 3691299, 3691510, 3691720, 3691788, 3691889, 3691966, 3692108, 3692184, 3692188, 3692272, 3692328, 3692372, 3692632, 3692899, 3692992, 3693020, 3693030, 3693197, 3693230, 3693231, 3693242, 3693253, 3693272, 3693409, 3693496, 3718061, 3729364, 3733641, 3775914, 3785109, 3792528, 3804869, 3830194, 3840450, 3944650, 3947363, 3961565, 3962848, 3962921, 3963067, 3963917, 3964105, 3966233, 3966460, 3967293, 3967462, 3967467, 3968312, 3968763, 3969008, 3969347, 3969776, 3969805, 3970664, 3970721, 3971102, 3971107, 3971865, 3972032, 3972164, 3972179, 3972249, 3972707, 3972877, 3972941, 3973200, 3973465, 4107793, 4119247, 4191756, 4218906, 4227522, 4322920, 4383124, 4410578, 4410733, 4421141, 4452169, 4509015, 4562144, 4705399, 4705425, 4705444, 4706768, 4708102, 4708106, 4708990, 4709205, 4710247, 4711411, 4712748, 4713337, 4714121, 4716786, 4717244, 4717334, 4717665, 4717738, 4718892, 4719065, 4735503, 4747945, 4769175, 4853970, 4897022, 4907883, 4919626, 4923321, 4985390, 5034097, 5056816, 5075551, 5075831, 5075949, 5076239, 5076382, 5076450, 5076480, 5076503, 5076600, 5076615, 5076646, 5076722, 5076747, 5076844, 5076874, 5077176, 5077274, 5077310, 5077450, 5077466, 5077488, 5077730, 5077763, 5077764, 5077789, 5077868, 5077966, 5078142, 5078286, 5078301, 5078314, 5078625, 5078671, 5078690, 5078735, 5078882, 5078951, 5078999, 5079005, 5079022, 5079392, 5079731, 5079787, 5079807, 5079978, 5080044, 5080069, 5080234, 5080432, 5080783, 5080935, 5080959, 5081121, 5081144, 5081638, 5081719, 5081823, 5082136, 5082172, 5082205, 5082210, 5082377, 5082533, 5082716, 5120504, 5120944, 5134162, 5181228, 5186230, 5186294, 5205690, 5215514, 5231684, 5356289, 5366415, 5403754, 5438203, 5467903, 5491686, 5494808, 5497865, 5523397, 5554205, 5610101, 5614351, 5621365, 5621641, 5649603, 5680898, 5854296, 5882534, 5924409, 5951840, 6018239, 6030303, 6039581, 6177433, 6177453, 6177521, 6177644, 6177812, 6177954, 6178065, 6178088, 6178181, 6178220, 6178393, 6178695, 6178939, 6179022, 6179073, 6179085, 6179099, 6179147, 6179295, 6179297, 6179644, 6179894, 6179954, 6179995, 6180024, 6180181, 6180280, 6180383, 6180603, 6180801, 6180859, 6180968, 6180976, 6181361, 6181466, 6181550, 6181772, 6181893, 6182088, 6182153, 6182324, 6182335, 6182469, 6182501, 6182503, 6182526, 6182908, 6182916, 6183030, 6183159, 6183231, 6183266, 6183302, 6183399, 6183973, 6183976, 6184481, 6184747, 6184949, 6185361, 6185362, 6185409, 6185661, 6185810, 6185833, 6185866, 6185871, 6185888, 6185911, 6186181, 6186251, 6186300, 6186347, 6186624, 6186746, 6186892, 6187018, 6187177, 6187210, 6187363, 6187474, 6187721, 6187744, 6187781, 6187782, 6188011, 6188148, 6188375, 6188392, 6188503, 6188528, 6188548, 6188645, 6188657, 6188929, 6188942, 6188988, 6189267, 6189453, 6189490, 6189536, 6189604, 6189636, 6189844, 6189884, 6189888, 6189914, 6190002, 6190005, 6190076, 6190270, 6190334, 6190399, 6190420, 6190543, 6190583, 6190602, 6190640, 6190827, 6190902, 6191034, 6191082, 6191083, 6191093, 6191168, 6191239, 6191240, 6191322, 6191387, 6212679, 6251550, 6309850, 6338508, 6372242, 6376613, 6383928, 6418035, 6429931, 6431163, 6449715, 6456158, 6460664, 6519826, 6520923, 6529883, 6554442, 6652031, 6659582, 6662449, 6677329, 6765943, 6784866, 6806475, 6830689, 6956908]]
for train_index, test_index in kf.split(X):
  pipe.fit(X[train_index], y[train_index])
  predicted = pipe.predict(X[test_index])
  fmacro = fmacro + metrics.f1_score(y[test_index], predicted, average='macro')
  fmicro = fmicro + metrics.f1_score(y[test_index], predicted, average='micro')
print("F-macro: ",fmacro/5)
print("F-micro: ",fmicro/5)

KeyError: ignored